In [1]:
from copy import deepcopy

import bw2data
import pandas as pd

from enbios.base.experiment import Experiment
from enbios.bw2.util import report
from enbios.models.experiment_base_models import ExperimentData

In [2]:
# get an overview of brightway projects and databases
report()

Project: default
['db']
Project: ecoinvent_391
['biosphere3', 'ecoinvent_391_cutoff']
Project: supply chain graph
['bike']
Project: nonlinear-method-test
['db']


In [3]:
# select the brightway project and database (e.g. some ecoinvent database)
PROJECT_NAME = "ecoinvent_391"
DATABASE = "ecoinvent_391_cutoff"

bw2data.projects.set_current(PROJECT_NAME)
db = bw2data.Database(DATABASE)

# Simple example experiment
In this first simple example we calculate the impact of 4 wind turbine activities in spain, using 2 different methods.

In [5]:
wind_turbines_spain = db.search(
    "electricity production, wind, 1-3MW turbine, onshore", filter={"location": "ES"}
)[:2]
wind_turbines_spain

Excluding 319 filtered results


['electricity production, wind, >3MW turbine, onshore' (kilowatt hour, ES, None),
 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, ES, None)]

In [6]:
# for the experiment we need to specify some (even if very simple) hierarchy made out of activity nodes at the bottom. 
# Activity notes
# we are using the brightway adapter, that is integrated into enbios.

experiment_activities = []

for activity in wind_turbines_spain:
    experiment_activities.append(
        {"name": activity["name"], "adapter": "bw", "config": {
            "code": activity["code"]
        }}
    )

In [7]:
# we can modify the output of the activities, by default it is the reference product (1 of the activity unit)
experiment_activities[0]["config"]["default_output"] = {"unit": "kilowatt_hour", "magnitude": 3}
experiment_activities

[{'name': 'electricity production, wind, >3MW turbine, onshore',
  'adapter': 'bw',
  'config': {'code': '0d48975a3766c13e68cedeb6c24f6f74',
   'default_output': {'unit': 'kilowatt_hour', 'magnitude': 3}}},
 {'name': 'electricity production, wind, 1-3MW turbine, onshore',
  'adapter': 'bw',
  'config': {'code': 'ed3da88fc23311ee183e9ffd376de89b'}}]

In [8]:
hierarchy = {
    "name": "root",
    "aggregator": "sum",
    "children": experiment_activities
}

In [9]:
# select 2 random methods and convert them into the form for enbios2
experiment_methods = [bw2data.methods.random() for _ in range(2)]

experiment_methods = {"_".join(m): m for m in experiment_methods}
experiment_methods

{'EF v3.1 no LT_human toxicity: carcinogenic no LT_comparative toxic unit for human (CTUh) no LT': ('EF v3.1 no LT',
  'human toxicity: carcinogenic no LT',
  'comparative toxic unit for human (CTUh) no LT'),
 'EF v3.1 no LT_photochemical oxidant formation: human health no LT_tropospheric ozone concentration increase no LT': ('EF v3.1 no LT',
  'photochemical oxidant formation: human health no LT',
  'tropospheric ozone concentration increase no LT')}

In [10]:
# alternatively, we could just specify two methods
experiment_methods = {
    "GWP1000": (
        "ReCiPe 2016 v1.03, midpoint (H)",
        "climate change",
        "global warming potential (GWP1000)",
    ),
    "FETP": (
        "ReCiPe 2016 v1.03, midpoint (H)",
        "ecotoxicity: freshwater",
        "freshwater ecotoxicity potential (FETP)",
    )
}

In [11]:
# let's store the raw data, because we want to modify it later
simple_raw_data = {
    "adapters": [
        {
            "adapter_name": "brightway-adapter",
            "config": {
                "bw_project": PROJECT_NAME
            },
            "methods": experiment_methods
        }],
    "hierarchy": hierarchy
}

# make a first validation of the experiment data
exp_data = ExperimentData(**simple_raw_data)
exp_data.model_dump(exclude_unset=True)

{'adapters': [{'adapter_name': 'brightway-adapter',
   'config': {'bw_project': 'ecoinvent_391'},
   'methods': {'GWP1000': ('ReCiPe 2016 v1.03, midpoint (H)',
     'climate change',
     'global warming potential (GWP1000)'),
    'FETP': ('ReCiPe 2016 v1.03, midpoint (H)',
     'ecotoxicity: freshwater',
     'freshwater ecotoxicity potential (FETP)')}}],
 'hierarchy': {'name': 'root',
  'aggregator': 'sum',
  'children': [{'name': 'electricity production, wind, >3MW turbine, onshore',
    'config': {'code': '0d48975a3766c13e68cedeb6c24f6f74',
     'default_output': {'unit': 'kilowatt_hour', 'magnitude': 3}},
    'adapter': 'bw'},
   {'name': 'electricity production, wind, 1-3MW turbine, onshore',
    'config': {'code': 'ed3da88fc23311ee183e9ffd376de89b'},
    'adapter': 'bw'}]}}

In [12]:
# create experiment object. This will validate the activities, their outputs, the methods and the scenarios.
simple_experiment: Experiment = Experiment(simple_raw_data)

In [13]:
print(simple_experiment.info())

Experiment: 
Activities: 2
root - sum-aggregator
 electricity production, wind, >3MW turbine, onshore - brightway-adapter
 electricity production, wind, 1-3MW turbine, onshore - brightway-adapter
Methods: 2
 bw.GWP1000
 bw.FETP
Hierarchy (depth): 2
Scenarios: 1


## Running the experiment

In [14]:
# run all scenarios at once, Results will be returned as dictionary
# <scenario_name : str> : <scenario_result : dict>
# <scenario_result> : dict represents the root node of the results tree, with the following keys:
# name, output, results, children
# where results, is a dictionary of impacts to dict: unit, amount | multi_amount
# The first and only positional parameter 'results_as_dict' can be set to False, to get the tree unserialized as :  BasicTreeNode[ScenarioResultNodeData]
results = simple_experiment.run()

2024-01-09 11:58:52,060 - enbios.demos.enbios.base - INFO - Running scenario 'default scenario'


# Result
The result is a dictionary of scenario names, where for each scenario we have a tree (representing the activity hierarchy). Each node (`BasicTreeNode`) in the tree has a `data` object, which is of the type `ScenarioResultNodeData`, which have the fields `output`, `result`.

In [15]:
results

{'default scenario': {'name': 'root',
  'results': {'GWP1000': {'unit': 'kg CO2-Eq', 'amount': 0.09015601391278945},
   'FETP': {'unit': 'kg 1,4-DCB-Eq', 'amount': 0.1572424802402813}},
  'output': {'unit': 'kilowatt_hour', 'amount': 4.0},
  'children': [{'name': 'electricity production, wind, >3MW turbine, onshore',
    'results': {'GWP1000': {'unit': 'kg CO2-Eq', 'amount': 0.0749124025799977},
     'FETP': {'unit': 'kg 1,4-DCB-Eq', 'amount': 0.15081687216577483}},
    'output': {'unit': 'kilowatt_hour', 'amount': 3.0}},
   {'name': 'electricity production, wind, 1-3MW turbine, onshore',
    'results': {'GWP1000': {'unit': 'kg CO2-Eq',
      'amount': 0.015243611332791748},
     'FETP': {'unit': 'kg 1,4-DCB-Eq', 'amount': 0.0064256080745064845}},
    'output': {'unit': 'kilowatt_hour', 'amount': 1.0}}]}}

In [16]:
simple_experiment.get_scenario("default scenario").result_tree.children[0].data

ScenarioResultNodeData(output=EnbiosQuantity(unit='kilowatt_hour', amount=3.0), results={'GWP1000': ResultValue(unit='kg CO2-Eq', amount=0.0749124025799977, multi_amount=[]), 'FETP': ResultValue(unit='kg 1,4-DCB-Eq', amount=0.15081687216577483, multi_amount=[])}, adapter='bw', aggregator=None)

In [17]:
# we can dump the results into a csv file
simple_experiment.results_to_csv("test.csv")
pd.read_csv("test.csv").fillna("")

,scenario,lvl_0,lvl_1,GWP1000_unit,GWP1000_amount,FETP_unit,FETP_amount,output_unit,output_amount
0,default scenario,root,,kg CO2-Eq,0.090156,"kg 1,4-DCB-Eq",0.157242,kilowatt_hour,4.0
1,,,"electricity production, wind, >3MW turbine, on...",kg CO2-Eq,0.074912,"kg 1,4-DCB-Eq",0.150817,kilowatt_hour,3.0
2,,,"electricity production, wind, 1-3MW turbine, o...",kg CO2-Eq,0.015244,"kg 1,4-DCB-Eq",0.006426,kilowatt_hour,1.0


In [18]:
simple_experiment.scenarios[0].result_to_dict()

{'name': 'root',
 'results': {'GWP1000': {'unit': 'kg CO2-Eq', 'amount': 0.09015601391278945},
  'FETP': {'unit': 'kg 1,4-DCB-Eq', 'amount': 0.1572424802402813}},
 'output': {'unit': 'kilowatt_hour', 'amount': 4.0},
 'children': [{'name': 'electricity production, wind, >3MW turbine, onshore',
   'results': {'GWP1000': {'unit': 'kg CO2-Eq', 'amount': 0.0749124025799977},
    'FETP': {'unit': 'kg 1,4-DCB-Eq', 'amount': 0.15081687216577483}},
   'output': {'unit': 'kilowatt_hour', 'amount': 3.0}},
  {'name': 'electricity production, wind, 1-3MW turbine, onshore',
   'results': {'GWP1000': {'unit': 'kg CO2-Eq',
     'amount': 0.015243611332791748},
    'FETP': {'unit': 'kg 1,4-DCB-Eq', 'amount': 0.0064256080745064845}},
   'output': {'unit': 'kilowatt_hour', 'amount': 1.0}}]}

## Add a technology hierarchy (dendrogram) 
Let's now add a few more activities to the experiment and create a hierarchy of activities.

In [14]:
solar_spain = db.search("solar", filter={"location": "ES"})[:2]
solar_spain

Excluding 465 filtered results


['electricity production, solar tower power plant, 20 MW' (kilowatt hour, ES, None),
 'electricity production, solar thermal parabolic trough, 50 MW' (kilowatt hour, ES, None)]

In [15]:
experiment_activities_solar = []
for activity in solar_spain:
    experiment_activities_solar.append(
        {"name": activity["name"], "adapter": "bw", "config": {
            "code": activity["code"]
        }}
    )
    
hierarchy_raw_data = deepcopy(simple_raw_data)

hierarchy_raw_data["hierarchy"] = {
    "name": "root",
    "aggregator": "sum",
    "children": [{
        "name": "wind",
        "aggregator": "sum",
        "children": experiment_activities
    },
        {
            "name": "solar",
            "aggregator": "sum",
            "children": experiment_activities_solar
        }
    ]
}

In [16]:
hierarchy_experiment: Experiment = Experiment(hierarchy_raw_data)

# Run the 2nd experiment

In [17]:
hierarchy_experiment.run()

2024-01-08 15:57:44,257 - enbios.demos.enbios.base - INFO - Running scenario 'default scenario'


{'default scenario': {'name': 'root',
  'results': {'GWP1000': {'unit': 'kg CO2-Eq', 'amount': 0.1925567846170288},
   'FETP': {'unit': 'kg 1,4-DCB-Eq', 'amount': 0.1635778761202299}},
  'output': {'unit': 'kilowatt_hour', 'amount': 6.0},
  'children': [{'name': 'wind',
    'results': {'GWP1000': {'unit': 'kg CO2-Eq',
      'amount': 0.09015601391278945},
     'FETP': {'unit': 'kg 1,4-DCB-Eq', 'amount': 0.1572424802402813}},
    'output': {'unit': 'kilowatt_hour', 'amount': 4.0},
    'children': [{'name': 'electricity production, wind, >3MW turbine, onshore',
      'results': {'GWP1000': {'unit': 'kg CO2-Eq',
        'amount': 0.0749124025799977},
       'FETP': {'unit': 'kg 1,4-DCB-Eq', 'amount': 0.15081687216577483}},
      'output': {'unit': 'kilowatt_hour', 'amount': 3.0}},
     {'name': 'electricity production, wind, 1-3MW turbine, onshore',
      'results': {'GWP1000': {'unit': 'kg CO2-Eq',
        'amount': 0.015243611332791748},
       'FETP': {'unit': 'kg 1,4-DCB-Eq', 'amount'

In [18]:
# print(json.dumps((exp.scenarios[0].result_to_dict()), indent=2))
hierarchy_experiment.scenarios[0].results_to_csv(
    "test.csv", level_names=["root", "technology", "activity"]
)
pd.read_csv("test.csv").fillna("")

,root,technology,activity,GWP1000_amount (kg CO2-Eq),"FETP_amount (kg 1,4-DCB-Eq)",output_unit,output_amount
0,root,,,0.192557,0.163578,kilowatt_hour,6.0
1,,wind,,0.090156,0.157242,kilowatt_hour,4.0
2,,,"electricity production, wind, >3MW turbine, on...",0.074912,0.150817,kilowatt_hour,3.0
3,,,"electricity production, wind, 1-3MW turbine, o...",0.015244,0.006426,kilowatt_hour,1.0
4,,solar,,0.102401,0.006335,kilowatt_hour,2.0
5,,,"electricity production, solar tower power plan...",0.048205,0.003242,kilowatt_hour,1.0
6,,,"electricity production, solar thermal paraboli...",0.054196,0.003093,kilowatt_hour,1.0


## Create several scenarios

In [20]:
from typing import Optional
from random import randint

def create_normal_scenario():
    return {
        "name":"normal scenario",
        "activities": {act: ("kilowatt_hour", 1) for act in hierarchy_experiment.activities_names}
    }

def create_random_scenario(scneario_name: Optional[str] = None):
    return {
        "name": scneario_name,
        "activities": {act: ("kilowatt_hour", randint(1, 10)) for act in hierarchy_experiment.activities_names}
    }

scenarios_raw_data = deepcopy(hierarchy_raw_data)

scenarios_raw_data["scenarios"] = [create_normal_scenario(),create_random_scenario(), create_random_scenario()]

scenarios_raw_data["scenarios"]

[{'name': 'normal scenario',
  'activities': {'electricity production, wind, >3MW turbine, onshore': ('kilowatt_hour',
    1),
   'electricity production, wind, 1-3MW turbine, onshore': ('kilowatt_hour',
    1),
   'electricity production, solar tower power plant, 20 MW': ('kilowatt_hour',
    1),
   'electricity production, solar thermal parabolic trough, 50 MW': ('kilowatt_hour',
    1)}},
 {'name': None,
  'activities': {'electricity production, wind, >3MW turbine, onshore': ('kilowatt_hour',
    5),
   'electricity production, wind, 1-3MW turbine, onshore': ('kilowatt_hour',
    9),
   'electricity production, solar tower power plant, 20 MW': ('kilowatt_hour',
    8),
   'electricity production, solar thermal parabolic trough, 50 MW': ('kilowatt_hour',
    3)}},
 {'name': None,
  'activities': {'electricity production, wind, >3MW turbine, onshore': ('kilowatt_hour',
    2),
   'electricity production, wind, 1-3MW turbine, onshore': ('kilowatt_hour',
    10),
   'electricity product

In [21]:
scenarios_experiment = Experiment(scenarios_raw_data)

 ## Run the experiment for the 3rd time
This time will likely take some more time since we need to run 2 scenarios. 

In [32]:
_ = scenarios_experiment.run()

2024-01-08 16:27:02,460 - enbios.demos.enbios.base - INFO - Running scenario 'normal scenario'


In [33]:
scenarios_experiment.scenarios[0].results_to_csv(
    "s1.csv", level_names=["root", "technology", "activity"]
)
pd.read_csv("s1.csv").fillna("")

,root,technology,activity,GWP1000_amount (kg CO2-Eq),"FETP_amount (kg 1,4-DCB-Eq)",output_unit,output_amount
0,root,,,0.142615,0.063033,kilowatt_hour,4.0
1,,wind,,0.040214,0.056698,kilowatt_hour,2.0
2,,,"electricity production, wind, >3MW turbine, on...",0.024971,0.050272,kilowatt_hour,1.0
3,,,"electricity production, wind, 1-3MW turbine, o...",0.015244,0.006426,kilowatt_hour,1.0
4,,solar,,0.102401,0.006335,kilowatt_hour,2.0
5,,,"electricity production, solar tower power plan...",0.048205,0.003242,kilowatt_hour,1.0
6,,,"electricity production, solar thermal paraboli...",0.054196,0.003093,kilowatt_hour,1.0


We can also now run any new scenario configuration for the given experiment using `run_scenario_config`

In [24]:
new_random_scenario = create_random_scenario()
new_random_scenario["name"] = "new scenario"
new_results = scenarios_experiment.run_scenario_config(new_random_scenario)

2024-01-08 16:39:18,606 - enbios.demos.enbios.base - INFO - Running scenario 'new scenario'


In [5]:
import json

print(json.dumps(Experiment.get_builtin_adapters(False), indent=2))

{
  "simple-assignment-adapter": {
    "activity_indicator": "assign"
  },
  "brightway-adapter": {
    "activity_indicator": "bw"
  }
}


In [18]:
from pathlib import Path

# Experiment.get_builtin_adapters()
[print(json.dump(a.get_config_schemas(), Path("temp.json").open("w"), indent=2)) for a in hierarchy_experiment.adapters]

None


[None]

In [16]:
from json_schema_for_humans.generate import generate_from_filename, generate_from_schema, generate_schemas_doc
from json_schema_for_humans.generation_configuration import GenerationConfiguration

config = GenerationConfiguration(copy_css=False, expand_buttons=False,template_name="md")

generate_from_filename("temp.json", "schema_doc.md", config=config)
# generate_from_schema(json.dumps(hierarchy_experiment.adapters[0].get_config_schemas()))
# Your doc is now in a file named "schema_doc.html". Next to it, "schema_doc.min.js" was copied, but not "schema_doc.css"
# Your doc will contain a "Expand all" and a "Collapse all" button at the top


== Generating schema_doc.md ==
== Generated schema_doc.md in 0:00:00.020017 ==


## Inspecting the results

We can now do some transformations of the results. For that is useful to know how to retrieve is singular result from a scenario result. 
The result of a scenario is a tree structure, where the nodes `name`s are activity aliases or names defined in the hierarchy. With the function of BasicTreeNode.find_child_by_name we can directly access the result of a node.    

Following we transform the results into a dictionary of the following structure:
```json
{
    "activity_alias": {
        "method_alias": "[list of results for each scenario]"
    }
}
```

In [ ]:
all_results = {}
for activity in activity_aliases:
    all_results[activity] = {
        method_alias: [] for method_alias in scenarios_experiment.method_aliases
    }
    for scenario in scenarios_experiment.scenarios:
        activity_result = scenario.result_tree.find_child_by_name(activity)
        for method, score in activity_result.data.results.items():
            all_results[activity][method].append(score)

all_results

In [ ]:
distribution_raw_data = deepcopy(simple_raw_data)
distribution_raw_data["config"] = {"use_k_bw_distributions": 5}

In [ ]:
distribution_experiment = Experiment(distribution_raw_data)
results = distribution_experiment.run()

In [ ]:
results["default scenario"].data.distribution_results